In [1]:
import numpy as np
import tensorflow as tf
print(tf.VERSION)

1.8.0


In [10]:
xavier = tf.contrib.layers.xavier_initializer()
zeros = tf.zeros_initializer()
dim_hidden = 1024 #Dim of hidden layer
vocabulary_size = 0 #TODO : FIT VOCABULARY SIZE
embedding_size = 512 #Size of the word embeddings
annotation_L = 196 #Annotation vectors are of the form L*D
annotation_D = 512 
max_timesteps = 16
lamda = 1.0
dropout1 = True
dropout1_rate = 0.5
dropout2 = True
dropout2_rate = 0.5
consider_z = True
consider_y = True
batch_size=196

In [5]:
def initial_c0_h0(annotations):
    with tf.variable_scope('initial_c0_h0'):
        feature_mean = tf.reduce_mean(annotations,axis=1)
        c0 = tf.layers.dense(inputs=feature_mean,units=dim_hidden,activation=tf.nn.tanh,kernel_initializer=xavier)
        h0 = tf.layers.dense(inputs=feature_mean,units=dim_hidden,activation=tf.nn.tanh,kernel_initializer=xavier)
        return c0,h0 #N * dim_hidden both

In [9]:
#UNDER CONSTRUCTION
def soft_attention(annotations,h_prev,i):
    with tf.variable_scope('soft_attention',reuse=(i != 0)):
        #HANDLE WITH CARE
        h_extend = tf.expand_dims(e,axis=1) # N * 1 * h
        onemat = tf.ones(shape=[batch_size,annotation_L,1]) # N * L * 1
        h_large = tf.matmul(onemat,h_extend) # N * L * h
        joined = tf.reshape(tf.concat([annotations,h_large],axis=2),[-1,dim_hidden + annotation_D]) # N  L * D + h
        e = tf.reshape(tf.layers.dense(inputs=joined,units=1,activation=tf.nn.relu,kernel_initializer=xavier),[-1,annotation_L])
        #HANDLE WITH CARE ENDS
        alpha = tf.nn.softmax(e) #dimension N * L
        beta = tf.layers.dense(inputs=h_prev,units=1,activation=tf.nn.sigmoid) #N * 1
        alpha_exp = tf.expand_dims(alpha,axis=1) #N * 1 * L
        pre_gating = tf.reshape(tf.matmul(alpha,annotations),[-1,annotation_D]) # (N,1,L) * (N,L,D) = (N,1,D) reshaped to (N,D)
        z = tf.multiply(beta,pre_gating) #N * 1 and N * D pointwise multiplication 
        return z,alpha,beta

In [4]:
def get_annotation(img_features):
    with tf.variable_scope('get_annotation_vecs'):
        W = tf.get_variable('W',[annotation_D,annotation_D])
        features_flat = tf.reshape(img_features,[-1,annotation_D])
        features_proj = tf.matmul(features_flat,W)
        annotations = tf.reshape(features_proj,[-1,annotation_L,annotation_D])
        return annotations # N * L * D

In [8]:
def word_embedding(one_hot,i):
    with tf.variable_scope('word_embedding',reuse=(i != 0)):
        word_embedding = tf.get_variable('word_embedding',[vocabulary_size,embedding_size],initializer=tf.random_uniform_initializer(minval=-1.0,maxval=1.0))
        return tf.nn.embedding_lookup(word_embedding,one_hot)    

In [11]:
def get_logits(ey,h,z,training,i):
    with tf.variable_scope('get_logits',reuse=(i != 0)):
        logits = h
        if dropout1:
            logits = tf.layers.dropout(inputs=logits,rate=dropout1_rate,training=training)
        logits = tf.layers.dense(inputs=logits,units=dim_embed,activation=None,kernel_initializer=xavier)
        if consider_y:
            logits += ey
        if consider_z:
            logits += tf.layers.dense(inputs=logits,units=dim_embed,activation=None,use_bias=False,kernel_initializer=xavier)
        logits = tf.nn.tanh(logits)
        if dropout2:
            logits = tf.layers.dropout(inputs=logits,rate=dropout2_rate,training=training)
        return tf.layers.dense(inputs=logits,units=dim_vocabulary,activation=None,kernel_initializer=xavier)